In [ ]:
import scipy.io
import pandas as pd
import numpy as np

# Load fMRI data
mat_nb = scipy.io.loadmat('E:/Brain_Develop_Knock/KARIM and ME/nback_fmri_power264.mat')
fmri_data = mat_nb.get('nback_fmri_power264')[0]

# Load phenotype data
pheno_data = pd.read_excel('E:/Brain_Develop_Knock/KARIM and ME/upenn_meta_cnb.xlsx')
subj_id_pheno = pheno_data['subject id']

# Load p264 template
p264_template = pd.read_excel('E:/Brain_Develop_Knock/KARIM and ME/PP264_template_subnet.xls', header=0)

# Initialize a list to store correlation features for each subnetwork
correlation_features_per_subnet = []

# Initialize lists to store correlation features for each subnet and age
correlation_features_per_subnet = [[] for _ in range(14)]  # Initialize a list for each subnet
age_in_month_subnet = []  # Initialize age_in_month_subnet as a list

# Iterate through subjects
for subj_count in range(len(mat_nb.get('nback_fmri_power264')[0])):
    print(f"SUBJ_COUNT is: {subj_count}")

    subj_id_nb = fmri_data[subj_count][0][0][0][0][0]

    # Check if subject is present in pheno_data
    if subj_id_nb in subj_id_pheno.values:
        # Get age of the subject in months
        subj_age_in_month = np.array(mat_nb.get('nback_fmri_power264')[0][subj_count][0][0][0][1])

        # Extract bold signals for all 264 regions
        bold_signals_all_regions = fmri_data[subj_count][2]

        # Compute Pearson correlation for all regions without transposing
        correlation_matrix_all_regions = np.corrcoef(bold_signals_all_regions)

        # Iterate through subnetwork columns
        for subnet_count in range(1, 15):
            # Get subnet indices from p264_template
            subnet_inds = p264_template[p264_template['subnet_inds'] == subnet_count]['Original_ROI'].values - 1

            print(f"subj_count: {subj_count}, subnet_count: {subnet_count}, subnet_inds: {subnet_inds}")

            # Extract values before the diagonal element for the current subnet
            values_before_diagonal = [
                correlation_matrix_all_regions[i, subnet_col] for subnet_col in subnet_inds for i in range(subnet_col)
            ]

            print(f"Shape of values_before_diagonal: {len(values_before_diagonal)}")

            # Append the list for the current subnet with values_before_diagonal
            correlation_features_per_subnet[subnet_count - 1].append(values_before_diagonal)

            print(f"Shape of correlation_features_per_subnet {subnet_count}: {np.array(correlation_features_per_subnet[subnet_count - 1]).shape}")

        # Store age in months for the current subject
        age_in_month_subnet.append(subj_age_in_month)

# Convert the lists of lists to a list of 2D NumPy arrays for further analysis
correlation_features_per_subnet = [np.array(features) for features in correlation_features_per_subnet]
age_in_month_subnet = np.array(age_in_month_subnet)

# Print shapes
for i, features in enumerate(correlation_features_per_subnet, start=1):
    print(f"Shape of correlation_features_per_subnet{i}: {features.shape}")

print(f"Shape of age_in_month_subnet: {age_in_month_subnet.shape}")


import numpy as np

# Extracting the ages from the 3D array
ages = y[:, :, 0]

# Creating y_cls based on the age threshold of 13 years
y_cls = np.where(ages > 14* 12, 1, 0)

# Counting the number of rows with y_cls = 1 and y_cls = 0
count_y_cls_1 = np.count_nonzero(y_cls == 1)
count_y_cls_0 = np.count_nonzero(y_cls == 0)

# Print the counts
print("Number of rows with y_cls = 1:", count_y_cls_1)
print("Number of rows with y_cls = 0:", count_y_cls_0)



# Function to normalize a group
def normalize_group(group):
    min_val = np.min(group, axis=0)
    max_val = np.max(group, axis=0)
    normalized_group = (group - min_val) / (max_val - min_val)
    return normalized_group

# Normalize each group and store in X_subnet_norm
X_subnet_norm = [normalize_group(group) for group in X_subnet]

# Print the shape of the first normalized group as an example
print("Shape of the first normalized group:", X_subnet_norm[0].shape)




from keras.layers import Input, Dense, Concatenate
from keras.models import Model
from sklearn.model_selection import train_test_split

# Extract the number of subjects
num_subjects = correlation_features_per_subnet[0].shape[0]

# Convert correlation_features_per_subnet to include a third dimension (batch_size, input_dim)
X_subnet = [subnet.reshape((num_subjects, -1, 1)) for subnet in correlation_features_per_subnet]

# Define the input layers for each subnet
subnet_inputs = []

num_features_list = [1008, 215, 735, 858, 5994, 754, 4867, 4693, 3789, 2951, 2037, 2832, 974, 3009]

for i, num_features in enumerate(num_features_list):
    subnet_input = Input(shape=(num_features,), name=f'input_subnet_{i + 1}')
    subnet_inputs.append(subnet_input)

# Dense layers for each subnet
subnet_dense_layers = [Dense(24, activation='elu')(subnet_input) for subnet_input in subnet_inputs]

# Concatenate the output of each subnet
concatenated_layer = Concatenate()(subnet_dense_layers)

# Dense layer with 32 neurons
dense_layer_32 = Dense(32, activation='elu')(concatenated_layer)

# Dense layer with 32 neurons
dense_layer_32 = Dense(64, activation='elu')(dense_layer_32)

# Dense layer with 32 neurons
dense_layer_32 = Dense(128, activation='elu')(dense_layer_32)

# Dense layer with 32 neurons
#dense_layer_32 = Dense(256, activation='relu')(dense_layer_32)


# Dense layer with 32 neurons
dense_layer_32 = Dense(256, activation='elu')(dense_layer_32)

# Output layer
output_layer = Dense(units=1, activation='sigmoid')(dense_layer_32)

# Create the model
model = Model(inputs=subnet_inputs, outputs=output_layer)


from keras.optimizers import Adam
from keras.metrics import BinaryAccuracy
from keras.losses import BinaryCrossentropy

# Assuming you have a binary classification problem

# Set the learning rate
learning_rate = 0.0001

# Set the gradient clipping value
clip_value = 0.5

# Create the Adam optimizer with learning rate
adam_optimizer = Adam(learning_rate=learning_rate, clipvalue=clip_value)


# Compile the model
model.compile(optimizer=adam_optimizer, loss='binary_crossentropy', metrics = 'accuracy')

# Display the model summary
model.summary()


num_epochs = 30
batch_size = 128

# Train the model with your data
model.fit(X_subnet_norm, np.array(y_cls), epochs=num_epochs, batch_size=batch_size, validation_split=0.1)

